In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [ ]:
toyota = pd.read_csv('/kaggle/input/used-car-dataset-ford-and-mercedes/toyota.csv')

toyota.head()

In [ ]:
toyota.isnull().sum()

In [ ]:
toyota.info()

In [ ]:
toyota.describe()

In [ ]:
skew = toyota.describe().T
skew['coef']=skew['std']/skew['mean']

skew

None have a particularly high coefficient of variation

In [ ]:
toyota.describe(include='O')

In [ ]:
cat_columns = ['model', 'transmission', 'fuelType']
num_columns = ['year', 'price', 'mileage', 'tax', 'mpg', 'engineSize']
num_columns_no_label = ['year', 'mileage', 'tax', 'mpg', 'engineSize']

In [ ]:
toyota['price'].value_counts().plot(kind='hist')
plt.title = 'Price Distribution'
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(20,30))
c=1
for i in toyota[num_columns].columns:
    plt.subplot(4,2,c)
    sns.distplot(toyota[i], color='blue')
    c=c+1
    plt.xlabel(i, fontsize=9)
    plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(15,30))
c=1
for i in toyota[cat_columns].columns:
    plt.subplot(3,1,c)
    sns.countplot(toyota[i])
    c=c+1
    plt.xlabel(i, fontsize=9)
    plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(toyota.corr(), annot=True, cmap='coolwarm')

EngineSize and Price have high positive correlation
(Year and Mileage) and (tax and mpg) and (price and mileage) are significantly negatively correlated

In [ ]:
X=toyota.drop(['price'], axis=1)
Y=toyota['price']

In [ ]:
toyota2 = pd.get_dummies(toyota, columns=cat_columns)

In [ ]:
toyota2.head()

In [ ]:
toyota2.info()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

scaler=StandardScaler()

x_train= scaler.fit_transform(x_train[num_columns_no_label])
x_test= scaler.transform(x_test[num_columns_no_label])

In [ ]:
X=toyota2.drop(['price'], axis=1)
Y=toyota2['price']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor

In [ ]:
lin_model = LinearRegression()
lin_model.fit(x_train, y_train)
lin_pred = lin_model.predict(x_test)
print('Train Accuracy: %f' % lin_model.score(x_train, y_train))
print('Test Accuracy: %f' % lin_model.score(x_test, y_test))

In [ ]:
from sklearn.model_selection import ShuffleSplit, cross_val_score, GridSearchCV

CV=ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

I'm going to make a function to find the best model

In [ ]:
def find_best(X,y):
    algos = {
        'linear': {
            'model': LinearRegression(),
            'params': {
                'normalize': [True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2], 
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion': ['mse','friedman_mse'], 
                'splitter': ['best', 'random']
            }
        }
    }
    scores=[]
    cv=ShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    for algo_name, config in algos.items():
        gs = GridSearchCV(config['model'], config['params'], cv=cv, return_train_score= False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })
    return pd.DataFrame(scores, columns=['model', 'best_score', 'best_params'])



In [ ]:
find_best(X,Y)

In [ ]:
final_model = DecisionTreeRegressor()

final_model.fit(x_train, y_train)
final_pred = final_model.predict(x_test)
print('Train Accuracy: %f' % final_model.score(x_train, y_train))
print('Test Accuracy: %f' % final_model.score(x_test, y_test))

In [ ]:
print('R2 Score: %f' % r2_score(y_test, final_pred)) 

In [ ]:
plt.scatter(y_test, final_pred)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')

z=np.polyfit(y_test, final_pred, 1)
p = np.poly1d(z)
plt.plot(y_test, p(y_test), color='magenta')
plt.show()